In [3]:
import os
import tweepy
from dotenv import load_dotenv

In [4]:
load_dotenv()

consumer_key = os.environ["API_KEY"]
consumer_secret = os.environ["API_KEY_SECRET"]
access_token = os.environ["ACCESS_TOKEN"]
access_token_secret = os.environ["ACCESS_TOKEN_SECRET"]
bearer_token = os.environ["BEARER_TOKEN"]


client = tweepy.Client(
  bearer_token=bearer_token,
  consumer_key=consumer_key,
  consumer_secret=consumer_secret,
  access_token=access_token,
  access_token_secret=access_token_secret,
  wait_on_rate_limit=True
)

In [96]:
def extraer_cuentas_usuarios(lista_cuentas, lista_tweets, response):
    #for i in range(len(response.data)):
    for i in range(len(response.includes['users'])):
        #lista_cuentas.append(response.includes['users'][i].username)
        lista_cuentas.append(response.includes['users'][i])
        lista_tweets.append(response.data[i].text)

    print(len(lista_cuentas))

In [97]:
# Search Recent Tweets
# This endpoint/method returns Tweets from the last seven days

#query = 'depresión yo tengo'
#query = 'Petro'
#query = 'Saben? A veces me pongo en los zapatos de Gustavo Petro y hasta me compadezco de él'
query = 'eafit'

# The method returns a Response object, a named tuple with data, includes,
# errors, and meta fields

cuentas_usuarios = []
tweets = []
for response in tweepy.Paginator(client.search_recent_tweets,
                                query,
                                tweet_fields=['id', 'author_id', 'text'],
                                user_fields=['id', 'username', 'location'],
                                expansions='author_id',
                                max_results=100, limit=1):#.flatten(limit=100):
    extraer_cuentas_usuarios(cuentas_usuarios, tweets, response)



82


In [98]:
print(len(cuentas_usuarios))

82


In [ ]:
print(tweets)

In [ ]:
for usuario in cuentas_usuarios:
    print(usuario.location)

In [101]:
usuarios = []
ids_usuarios = []
for usuario in cuentas_usuarios:
    if (usuario.location is not None):
        if ('Medellin' in usuario.location) or ('Medellín' in usuario.location) or ('medellin' in usuario.location) or ('medellín' in usuario.location):
            usuarios.append(usuario.username)
            ids_usuarios.append(usuario.id)

In [102]:
print(usuarios)
print(ids_usuarios)

['santiagosilvaj', 'NaNo__11', 'EAFIT', 'jnicolasosorio', 'Vyral_News_', 'FEG_EAFIT', 'TheImpishTerror', 'vdavidayala', 'labrinth09', 'XLordValdomeroX', 'RepresEAFIT', 'sjrm59', 'mariosuarezsuar', 'sexcol2020', 'camilo2802']
[147626104, 234089444, 41704249, 322308888, 1177799339764256768, 2385510302, 4379250803, 883789447652937728, 1423415860929118211, 805817971, 394312666, 881251408238051328, 190627315, 1093233540962111488, 2910380363]


In [103]:
usuarios_globales = set(usuarios)

In [104]:
print(usuarios_globales)

{'sexcol2020', 'jnicolasosorio', 'sjrm59', 'RepresEAFIT', 'NaNo__11', 'Vyral_News_', 'EAFIT', 'FEG_EAFIT', 'vdavidayala', 'camilo2802', 'XLordValdomeroX', 'mariosuarezsuar', 'TheImpishTerror', 'santiagosilvaj', 'labrinth09'}


In [105]:
def extraer_tweets_usuarios(id, diccionario, response):
    #for i in range(len(response.data)):
    lista_tweets = []
    print(len(response.data))
    for i in range(len(response.data)):
        lista_tweets.append(response.data[i].text)

    diccionario[id] = lista_tweets

In [106]:
users_tweets = dict()
for id_usuario in ids_usuarios:
    for response in tweepy.Paginator(client.get_users_tweets,
                                    id_usuario,
                                    tweet_fields=['id', 'author_id', 'text'],
                                    user_fields=['id', 'username', 'location'],
                                    expansions='referenced_tweets.id',
                                    #expansions='author_id',
                                    max_results=100, limit=1):#.flatten(limit=100):
        extraer_tweets_usuarios(id_usuario, users_tweets, response)
        #print(response.includes['tweets'])

        print(len(response.includes))
        for i in range(len(response.data)):
            if response.data[i]['referenced_tweets']:
                print(response.data[i]['referenced_tweets'][0]['type'])
            else:
                print(response.data[i])
        break
    break

100
1
retweeted
retweeted
replied_to
retweeted
retweeted
retweeted
retweeted
replied_to
replied_to
replied_to
replied_to
replied_to
replied_to
La foto del telescopio Jame Webb que tanto hemos compartido por acá, aparte de preciosa y además de un avance científico extraordinario es un buen recordatorio de lo minúsculos que somos respecto al universo.
Un recordatorio de la necesaria humildad cósmica. https://t.co/NlZKy8IpCS
retweeted
retweeted
retweeted
replied_to
replied_to
replied_to
replied_to
retweeted
Las columnas más satisfactorias de escribir son las más ñoñas. La de hoy lo es. Pero prometo que es interesante. 
Sobre los estoicos y la misión que deberíamos compartir de ampliar nuestros círculos de preocupación.
Gracias por leer.
https://t.co/tekBwoB85h
retweeted
retweeted
retweeted
retweeted
retweeted
retweeted
retweeted
retweeted
retweeted
Artículo: Una propuesta de criterios de selección de instrumentos para intervenciones en cultura ciudadana, aproximación al problema ético. ht

In [ ]:
print(users_tweets[827918863220228097][12])

In [ ]:


# #print(tweets)

# # Each Tweet object has default ID and text fields
# for tweet in tweets:
#     print(tweet.id)
#     print(tweet)
#     print(tweet.text)